# Preparations

## Loading packages

In [ ]:
%pip install qiskit qiskit_aer

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 12.9 MB/s eta 0:00:00
  Created wheel for qiskit: filename=qiskit-0.44.0-py3-none-any.whl size=7620 sha256=1c3f791f917f2ae34f7f8a0451d20d70bd20a183e70cf791355a872063234a02
  Stored in directory: /root/.

In [ ]:
from itertools import combinations, permutations, groupby
import networkx as nx
from networkx.algorithms.approximation import one_exchange
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import tqdm
from multiprocessing import Pool

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, pauli_error, ReadoutError, thermal_relaxation_error, depolarizing_error
from qiskit.quantum_info import Statevector, DensityMatrix

## Graph generator

In [ ]:
"""
generate all subsets
"""
def subsets(ls):
    res = []
    def dfs(i, subset):
        # base case
        if i == len(ls):
            res.append(subset.copy())
            return

        dfs(i + 1, subset)

        subset.append(ls[i])
        dfs(i + 1, subset)
        subset.pop()

    dfs(0, [])
    return res

"""
check if graph g1 is isomorphic to any graph in the list
"""
def is_unique(ls, g1):
    if not ls:
        return True
    for g2 in ls:
        if nx.is_isomorphic(g1, g2):
            return False
    return True

"""
returns {graph: multiplicity}
"""
def generate_connected_graphs(n):  # n vertices
    all_edges = list(combinations(range(n), 2))
    edge_subsets = [i for i in subsets(all_edges) if len(i) >= n - 1] # rule out the obvious cases
    res = {}

    for subset in edge_subsets:
        G = nx.Graph()
        G.add_nodes_from(range(n))
        G.add_edges_from(subset)

        if nx.is_connected(G):
            if is_unique(list(res.keys()), G):
                res[G] = 1
            else:
                for k in res.keys():
                    if nx.is_isomorphic(k, G):
                        res[k] += 1
                        break

    return res

In [ ]:
def ErdosEdwards(n, m):  # num of vertices and num of edges
    return int(np.ceil((2 * m + n - 1)/4))

def get_graph_data(G, m):
    GraphLaplacian = np.array(nx.laplacian_matrix(G).toarray())
    NumberOfVerticies, NumberOfEdges = G.number_of_nodes(), G.number_of_edges()
    threshold = ErdosEdwards(NumberOfEdges, NumberOfVerticies)

    maxdegvertex = np.where(np.diag(GraphLaplacian) == max(np.diag(GraphLaplacian)))[0][0]
    if maxdegvertex + 1 != NumberOfVerticies:
        GraphLaplacian[[maxdegvertex, -1], :] = GraphLaplacian[[-1, maxdegvertex], :]
        GraphLaplacian[:, [maxdegvertex, -1]] = GraphLaplacian[:, [-1, maxdegvertex]]
    QuadraticForm = GraphLaplacian[np.ix_(np.arange(NumberOfVerticies - 1), np.arange(NumberOfVerticies - 1))]

    # cut-function and MaxCut
    cuts = np.zeros(2 ** len(QuadraticForm), dtype=int)
    for x in range(1, 2 ** len(QuadraticForm)):
        conf = (x >> np.arange(len(QuadraticForm)))%2
        cuts[x] = np.matmul(conf, np.matmul(QuadraticForm, np.transpose(conf)))
    MaxCut = max(cuts)

    # cut disctibution and ratios
    random_chance, Lambda, dp = {i: 0 for i in range(MaxCut + 1)}, 0, pow(2, 1 - NumberOfVerticies)
    for _, cut in enumerate(cuts):
        random_chance[cut] += dp
        if cut >= threshold:
            Lambda += dp

    return [G, m, QuadraticForm, threshold, Lambda, random_chance]

## Simulator

In [ ]:
# L, x must be positive
def Chebyshev(L:  float, x: float):
    return np.cos(L * np.arccos(x)) if x <= 1 else np.cosh(L * np.arccosh(x))

# delta, Lambda must be in (0, 1)
def GroverProbability(delta: float, Lambda: float, L: float):
    return 1 - pow(delta * Chebyshev(L, Chebyshev(1 / L, 1 / delta) * np.sqrt(1 - Lambda)), 2)

# P, Lambda must be in (0, 1), N must be a positive integer
def GroverParameterOptimizer(Lambda: float, N: int):
    d = delta = pow(2, - N)
    P = 0
    while d < 1:
        if GroverProbability(d, Lambda, 3) >= P:
            delta, P = d, GroverProbability(d, Lambda, 3)
        d += pow(2, - N)

    return delta, P

In [ ]:
def Grover(data):
    G, m, QuadraticForm, threshold, Lambda, random_chance = data[0], data[1], data[2], data[3], data[4], data[5]
    delta, P = GroverParameterOptimizer(Lambda, 10)
    alpha = beta = - 2 * np.arctan(pow(1 - pow(np.cosh(np.arccosh(1 / delta) / 3), - 2), - 1/2) / np.sqrt(3))

    # quantum registers for the bit configurations
    QRegX = QuantumRegister(len(QuadraticForm), "x")

    NumberOfEdges = int((np.sum(QuadraticForm) + np.trace(QuadraticForm))/2)

    digits = 1 + int(np.ceil(np.log2(max(threshold, NumberOfEdges + 1 - threshold))))

    # quantum registers to digitize values
    QRegY = QuantumRegister(digits, "y")

    QC = QuantumCircuit(QRegX, QRegY)

    QC.h(QRegX[:] + QRegY[:])

    # adding threshold - offset; the offset could have gone to later parts of the code, but this it's cheaper this way
    theta = (threshold - 1/2 - (np.sum(QuadraticForm) + np.trace(QuadraticForm))/4) * np.pi
    for i, q in enumerate(reversed(QRegY)):
        QC.rz(theta - np.pi/2, q)
        theta /= 2

    # S_t (beta)
    for i, p in enumerate(QRegX):

        if np.sum(QuadraticForm[i]) != 0:
            theta = np.sum(QuadraticForm[i]) * np.pi
            for q in reversed(QRegY):
                QC.cx(p, q)
                theta /= 2
                QC.rz(theta, q)
                QC.cx(p, q)

        for j, r in enumerate(QRegX[i + 1:]):
            if QuadraticForm[i][i + 1 + j] != 0:
                QC.cx(p, r)
                theta = QuadraticForm[i][i + 1 + j] * np.pi
                for q in reversed(QRegY):
                    QC.cx(r, q)
                    theta /= 2
                    QC.rz(- theta, q)
                    QC.cx(r, q)
                QC.cx(p, r)

    for i, q in enumerate(reversed(QRegY[1:])):
        QC.h(q)
        for j, r in enumerate(QRegY[:len(QRegY) - 1 - i]):
            QC.cx(q, r)
            QC.rz(pow(2, i + j - digits) * np.pi, r)
        for r in QRegY[:len(QRegY) - 2 - i]:
            QC.cx(q, r)

    QC.rx(beta, QRegY[0])

    for i, q in enumerate(QRegY[1:]):
        for r in reversed(QRegY[:i]):
            QC.cx(q, r)
        theta = np.pi / 2
        for r in reversed(QRegY[:i+1]):
            theta /= 2
            QC.rz(- theta, r)
            QC.cx(q, r)
        QC.h(q)

    for i, p in enumerate(reversed(QRegX)):

        for j, r in enumerate(reversed(QRegX[len(QRegX) - i:])):
            if QuadraticForm[- i - 1][- j - 1] != 0:
                QC.cx(p, r)
                theta = QuadraticForm[- i - 1][- j - 1] * np.pi
                for q in reversed(QRegY):
                    theta /= 2
                    QC.cx(r, q)
                    QC.rz(theta, q)
                    QC.cx(r, q)
                QC.cx(p, r)

        if np.sum(QuadraticForm[- i - 1]) != 0:
            theta = np.sum(QuadraticForm[- i - 1]) * np.pi
            for q in reversed(QRegY):
                theta /= 2
                QC.cx(p, q)
                QC.rz(- theta, q)
                QC.cx(p, q)

        # S_s (alpha)
        QC.ry(np.pi/2, QRegX)
        QC.mcp(alpha, QRegX[1:], QRegX[0])
        QC.ry(- np.pi/2, QRegX)

        QC.save_statevector()
        simulator = AerSimulator(method="statevector")
        psi = Statevector(execute(QC, simulator).result().get_statevector())

        return [G, m, QuadraticForm, threshold, Lambda, random_chance, P, psi, QC.depth(), QC.count_ops()]

# Run computations

In [ ]:
graphs = generate_connected_graphs(5)
# Graph, multiplicity, QuadraticForm, threshold, Lambda, random_chance
graphs_data = [get_graph_data(G, m) for (G, m) in graphs.items()]

TypeError: ignored

In [ ]:
with Pool() as pool:
    results = list(tqdm.tqdm(pool.imap(Grover, graphs_data), total=len(graphs_data)))